In [6]:
from nltk import tokenize

In [7]:
import gensim
import logging
import re

logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

In [13]:
sentences = []
with open('Data/articles1.csv', 'rb') as f:
    l = 0
    for i,line in enumerate(f):
        l+=1
#         entry = line.split(",")
#         space, ID, title, publication, author, date, year, month, url, content = entry[0], entry[1], entry[2], entry[3], entry[4], entry[5], entry[6], entry[7], entry[8], entry[9:]
        
#         # formatting content text
#         content_text = re.sub(r"[  ]+", " ",(", ").join(content)).decode("utf8")
        
#         sentences += tokenize.sent_tokenize(content_text)
            
    print l

50007


In [ ]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""

    logging.info("reading file {0}...this may take a while".format(input_file))
    with open(input_file, 'rb') as f:
        documents = []
        
        for i, line in enumerate(f):
            entry = line.split(",")
            try:
                space, ID, title, publication, author, date, year, month, url, content = entry[0], entry[1], entry[2], entry[3], entry[4], entry[5], entry[6], entry[7], entry[8], entry[9:]
            except:
                print "FAILURE"
                continue
                
            content_text = re.sub(r"[  ]+", " ",(", ").join(content)).decode("utf8")
            sentences = tokenize.sent_tokenize(content_text)
            
            if (i % 5000 == 0):
                logging.info("read {0} reviews".format(i))
                    
            sentences = [gensim.utils.simple_preprocess(s) for s in sentences]  
            documents.append(sentences)

        return documents
            
documents = list(read_input('Data/articles1.csv'))
logging.info ("Done reading data file")

2018-12-12 11:58:16,594 : INFO : reading file Data/articles1.csv...this may take a while
2018-12-12 11:58:16,715 : INFO : read 0 reviews
2018-12-12 11:58:55,700 : INFO : read 5000 reviews
2018-12-12 11:59:23,557 : INFO : read 10000 reviews


In [15]:
obama_docs = []
full_documents = []
for doc in documents:
  
    text = [word for sentence in doc for word in sentence]

    full_documents.append(text)
    if "democrat" in text:
        obama_docs.append(doc)
        
print len(obama_docs)

2916


In [16]:
from gensim import corpora
dictionary = corpora.Dictionary(full_documents)
corpus = [dictionary.doc2bow(text) for text in full_documents]
import pickle
pickle.dump(corpus, open('dem_corpus.pkl', 'wb'))
dictionary.save('dem_dictionary.gensim')

2018-12-12 14:42:45,808 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-12-12 14:43:11,261 : INFO : adding document #10000 to Dictionary(98148 unique tokens: [u'fawn', u'biennials', u'biysk', u'vang', u'sonji']...)
2018-12-12 14:43:23,944 : INFO : adding document #20000 to Dictionary(119750 unique tokens: [u'fawn', u'biennials', u'biysk', u'vang', u'sonji']...)
2018-12-12 14:43:40,323 : INFO : adding document #30000 to Dictionary(136882 unique tokens: [u'fawn', u'biennials', u'biysk', u'vang', u'sonji']...)
2018-12-12 14:43:53,343 : INFO : adding document #40000 to Dictionary(154038 unique tokens: [u'fawn', u'biennials', u'biysk', u'vang', u'sonji']...)
2018-12-12 14:44:02,678 : INFO : adding document #50000 to Dictionary(166278 unique tokens: [u'fawn', u'biennials', u'biysk', u'vang', u'sonji']...)
2018-12-12 14:44:02,680 : INFO : built Dictionary(166278 unique tokens: [u'fawn', u'biennials', u'biysk', u'vang', u'sonji']...) from 50001 documents (total 30355595 cor

In [ ]:
#########

In [5]:
obama_docs = []
for doc in documents:
  
    text = [word for sentence in doc for word in sentence]

    if "obama" in text:
        obama_docs.append(text)
        
print len(obama_docs)

NameError: name 'documents' is not defined

In [82]:
import nltk

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ninawang/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [83]:

text_data = []
for text in obama_docs:
    tokens = [token for token in text if token not in en_stop]
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [get_lemma(token) for token in tokens]
    text_data.append(tokens)

In [90]:
c =0 
for text in text_data:
    if "the" in text:
        c+=1
print c
    

0


In [85]:
text_dictionary = corpora.Dictionary(text_data)
text_corpus = [text_dictionary.doc2bow(text) for text in text_data]

pickle.dump(text_corpus, open('corpus.pkl', 'wb'))
text_dictionary.save('text_dictionary.gensim')


2018-12-12 01:23:42,129 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-12-12 01:23:50,538 : INFO : adding document #10000 to Dictionary(52737 unique tokens: [u'fawn', u'tripolitan', u'herve', u'deferment', u'sowell']...)
2018-12-12 01:23:50,626 : INFO : built Dictionary(52983 unique tokens: [u'fawn', u'tripolitan', u'herve', u'deferment', u'sowell']...) from 10079 documents (total 3916088 corpus positions)
2018-12-12 01:24:34,685 : INFO : saving Dictionary object under text_dictionary.gensim, separately None
2018-12-12 01:24:35,407 : INFO : saved text_dictionary.gensim


In [93]:
import gensim

NUM_TOPICS = 7
ldamodel = gensim.models.ldamodel.LdaModel(text_corpus, num_topics = NUM_TOPICS, id2word=text_dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

2018-12-12 01:44:38,008 : INFO : using symmetric alpha at 0.142857142857
2018-12-12 01:44:38,009 : INFO : using symmetric eta at 0.142857142857
2018-12-12 01:44:38,028 : INFO : using serial LDA version on this node
2018-12-12 01:44:38,096 : INFO : running online (multi-pass) LDA training, 7 topics, 15 passes over the supplied corpus of 10079 documents, updating model once every 2000 documents, evaluating perplexity every 10079 documents, iterating 50x with a convergence threshold of 0.001000
2018-12-12 01:44:38,098 : INFO : PROGRESS: pass 0, at document #2000/10079
2018-12-12 01:44:41,064 : INFO : merging changes from 2000 documents into a model of 10079 documents
2018-12-12 01:44:41,238 : INFO : topic #3 (0.143): 0.012*"trump" + 0.008*"state" + 0.007*"would" + 0.005*"american" + 0.005*"president" + 0.005*"obama" + 0.004*"could" + 0.004*"unite" + 0.004*"house" + 0.004*"official"
2018-12-12 01:44:41,242 : INFO : topic #0 (0.143): 0.011*"state" + 0.010*"trump" + 0.008*"would" + 0.007*"pe

2018-12-12 01:44:51,709 : INFO : topic #5 (0.143): 0.013*"obama" + 0.010*"president" + 0.009*"white" + 0.009*"trump" + 0.007*"police" + 0.007*"house" + 0.006*"people" + 0.005*"story" + 0.004*"officer" + 0.004*"black"
2018-12-12 01:44:51,710 : INFO : topic #0 (0.143): 0.014*"state" + 0.009*"federal" + 0.008*"would" + 0.006*"obama" + 0.006*"administration" + 0.006*"government" + 0.006*"people" + 0.006*"school" + 0.005*"climate" + 0.005*"policy"
2018-12-12 01:44:51,713 : INFO : topic diff=0.798988, rho=0.408248
2018-12-12 01:44:51,716 : INFO : PROGRESS: pass 1, at document #2000/10079
2018-12-12 01:44:54,958 : INFO : merging changes from 2000 documents into a model of 10079 documents
2018-12-12 01:44:54,996 : INFO : topic #6 (0.143): 0.035*"trump" + 0.010*"president" + 0.008*"obama" + 0.008*"would" + 0.008*"state" + 0.007*"russia" + 0.006*"russian" + 0.006*"administration" + 0.006*"official" + 0.005*"intelligence"
2018-12-12 01:44:54,997 : INFO : topic #5 (0.143): 0.010*"obama" + 0.009*"w

2018-12-12 01:45:04,698 : INFO : topic #0 (0.143): 0.015*"state" + 0.010*"federal" + 0.009*"would" + 0.007*"administration" + 0.006*"school" + 0.006*"obama" + 0.006*"government" + 0.006*"policy" + 0.006*"people" + 0.005*"immigration"
2018-12-12 01:45:04,700 : INFO : topic #3 (0.143): 0.006*"company" + 0.005*"business" + 0.005*"years" + 0.005*"would" + 0.005*"change" + 0.005*"since" + 0.005*"climate" + 0.005*"obama" + 0.005*"first" + 0.004*"worker"
2018-12-12 01:45:04,703 : INFO : topic diff=0.582550, rho=0.377964
2018-12-12 01:45:04,705 : INFO : PROGRESS: pass 2, at document #2000/10079
2018-12-12 01:45:07,395 : INFO : merging changes from 2000 documents into a model of 10079 documents
2018-12-12 01:45:07,427 : INFO : topic #5 (0.143): 0.011*"obama" + 0.010*"white" + 0.009*"president" + 0.008*"police" + 0.007*"people" + 0.006*"house" + 0.006*"black" + 0.005*"officer" + 0.004*"medium" + 0.004*"story"
2018-12-12 01:45:07,429 : INFO : topic #0 (0.143): 0.015*"state" + 0.010*"would" + 0.01

2018-12-12 01:45:16,038 : INFO : topic #4 (0.143): 0.011*"state" + 0.010*"attack" + 0.010*"military" + 0.008*"nuclear" + 0.008*"force" + 0.008*"group" + 0.008*"syria" + 0.006*"unite" + 0.006*"weapon" + 0.005*"country"
2018-12-12 01:45:16,040 : INFO : topic #1 (0.143): 0.018*"obama" + 0.015*"president" + 0.009*"state" + 0.008*"american" + 0.007*"unite" + 0.007*"country" + 0.007*"court" + 0.006*"would" + 0.006*"people" + 0.006*"israel"
2018-12-12 01:45:16,042 : INFO : topic diff=0.482028, rho=0.353553
2018-12-12 01:45:16,045 : INFO : PROGRESS: pass 3, at document #2000/10079
2018-12-12 01:45:19,333 : INFO : merging changes from 2000 documents into a model of 10079 documents
2018-12-12 01:45:19,374 : INFO : topic #3 (0.143): 0.009*"company" + 0.006*"climate" + 0.006*"change" + 0.006*"business" + 0.005*"years" + 0.005*"would" + 0.005*"china" + 0.005*"percent" + 0.005*"since" + 0.004*"worker"
2018-12-12 01:45:19,376 : INFO : topic #0 (0.143): 0.016*"state" + 0.011*"would" + 0.010*"federal" 

2018-12-12 01:45:31,197 : INFO : topic #3 (0.143): 0.008*"climate" + 0.007*"change" + 0.007*"company" + 0.006*"business" + 0.005*"years" + 0.005*"since" + 0.005*"would" + 0.004*"obama" + 0.004*"percent" + 0.004*"first"
2018-12-12 01:45:31,199 : INFO : topic #4 (0.143): 0.012*"state" + 0.010*"attack" + 0.010*"military" + 0.009*"nuclear" + 0.008*"force" + 0.007*"syria" + 0.007*"group" + 0.007*"unite" + 0.006*"weapon" + 0.005*"country"
2018-12-12 01:45:31,201 : INFO : topic diff=0.427690, rho=0.333333
2018-12-12 01:45:31,206 : INFO : PROGRESS: pass 4, at document #2000/10079
2018-12-12 01:45:34,319 : INFO : merging changes from 2000 documents into a model of 10079 documents
2018-12-12 01:45:34,355 : INFO : topic #0 (0.143): 0.016*"state" + 0.011*"would" + 0.011*"federal" + 0.007*"administration" + 0.007*"health" + 0.007*"school" + 0.007*"republican" + 0.007*"government" + 0.006*"people" + 0.006*"policy"
2018-12-12 01:45:34,356 : INFO : topic #1 (0.143): 0.016*"obama" + 0.015*"president" +

...

2018-12-12 01:47:13,871 : INFO : topic #2 (0.143): 0.043*"trump" + 0.022*"clinton" + 0.012*"republican" + 0.009*"campaign" + 0.008*"president" + 0.008*"obama" + 0.008*"would" + 0.008*"hillary" + 0.008*"people" + 0.007*"donald"
2018-12-12 01:47:13,874 : INFO : topic #6 (0.143): 0.046*"trump" + 0.010*"president" + 0.009*"russia" + 0.008*"russian" + 0.007*"intelligence" + 0.007*"election" + 0.007*"government" + 0.007*"official" + 0.006*"report" + 0.006*"would"
2018-12-12 01:47:13,877 : INFO : topic #3 (0.143): 0.008*"climate" + 0.008*"change" + 0.007*"company" + 0.007*"business" + 0.006*"would" + 0.005*"since" + 0.005*"trump" + 0.005*"years" + 0.005*"percent" + 0.005*"obama"
2018-12-12 01:47:13,880 : INFO : topic #4 (0.143): 0.013*"state" + 0.009*"attack" + 0.009*"military" + 0.008*"nuclear" + 0.008*"unite" + 0.007*"force" + 0.007*"syria" + 0.007*"group" + 0.006*"china" + 0.006*"would"
2018-12-12 01:47:13,882 : INFO : topic #5 (0.143): 0.014*"obama" + 0.010*"president" + 0.008*"white" + 0

2018-12-12 01:47:28,287 : INFO : merging changes from 79 documents into a model of 10079 documents
2018-12-12 01:47:28,324 : INFO : topic #6 (0.143): 0.046*"trump" + 0.010*"president" + 0.009*"russia" + 0.008*"russian" + 0.007*"intelligence" + 0.007*"election" + 0.007*"official" + 0.007*"government" + 0.006*"report" + 0.006*"would"
2018-12-12 01:47:28,326 : INFO : topic #3 (0.143): 0.008*"climate" + 0.008*"change" + 0.007*"company" + 0.007*"business" + 0.006*"would" + 0.005*"trump" + 0.005*"since" + 0.005*"years" + 0.005*"percent" + 0.005*"obama"
2018-12-12 01:47:28,328 : INFO : topic #2 (0.143): 0.043*"trump" + 0.022*"clinton" + 0.012*"republican" + 0.009*"campaign" + 0.008*"president" + 0.008*"obama" + 0.008*"would" + 0.008*"hillary" + 0.008*"people" + 0.007*"donald"
2018-12-12 01:47:28,330 : INFO : topic #4 (0.143): 0.013*"state" + 0.009*"attack" + 0.009*"military" + 0.008*"nuclear" + 0.008*"unite" + 0.007*"force" + 0.007*"syria" + 0.007*"group" + 0.006*"china" + 0.006*"would"
2018-

2018-12-12 01:47:40,514 : INFO : PROGRESS: pass 13, at document #10079/10079
2018-12-12 01:47:40,628 : INFO : merging changes from 79 documents into a model of 10079 documents
2018-12-12 01:47:40,665 : INFO : topic #4 (0.143): 0.013*"state" + 0.009*"attack" + 0.009*"military" + 0.008*"unite" + 0.008*"nuclear" + 0.007*"force" + 0.007*"syria" + 0.007*"group" + 0.006*"china" + 0.006*"would"
2018-12-12 01:47:40,667 : INFO : topic #5 (0.143): 0.013*"obama" + 0.010*"president" + 0.008*"white" + 0.007*"people" + 0.007*"police" + 0.005*"black" + 0.005*"twitter" + 0.005*"house" + 0.005*"first" + 0.005*"medium"
2018-12-12 01:47:40,668 : INFO : topic #0 (0.143): 0.015*"state" + 0.011*"would" + 0.010*"federal" + 0.010*"republican" + 0.007*"government" + 0.007*"school" + 0.006*"congress" + 0.006*"administration" + 0.006*"health" + 0.006*"policy"
2018-12-12 01:47:40,671 : INFO : topic #6 (0.143): 0.046*"trump" + 0.010*"president" + 0.009*"russia" + 0.009*"russian" + 0.007*"intelligence" + 0.007*"ele

2018-12-12 01:47:52,537 : INFO : topic diff=0.107150, rho=0.223607
2018-12-12 01:47:52,830 : INFO : -7.607 per-word bound, 194.9 perplexity estimate based on a held-out corpus of 79 documents with 49341 words
2018-12-12 01:47:52,831 : INFO : PROGRESS: pass 14, at document #10079/10079
2018-12-12 01:47:52,961 : INFO : merging changes from 79 documents into a model of 10079 documents
2018-12-12 01:47:53,003 : INFO : topic #2 (0.143): 0.044*"trump" + 0.023*"clinton" + 0.012*"republican" + 0.009*"campaign" + 0.008*"president" + 0.008*"obama" + 0.008*"would" + 0.008*"hillary" + 0.008*"people" + 0.007*"donald"
2018-12-12 01:47:53,005 : INFO : topic #1 (0.143): 0.019*"obama" + 0.016*"president" + 0.009*"american" + 0.009*"state" + 0.008*"people" + 0.008*"administration" + 0.007*"unite" + 0.007*"country" + 0.007*"would" + 0.006*"policy"
2018-12-12 01:47:53,007 : INFO : topic #3 (0.143): 0.008*"climate" + 0.008*"change" + 0.007*"company" + 0.007*"business" + 0.006*"trump" + 0.006*"would" + 0.00

(0, u'0.015*"state" + 0.011*"would" + 0.010*"federal" + 0.010*"republican"')
(1, u'0.019*"obama" + 0.016*"president" + 0.009*"american" + 0.009*"state"')
(2, u'0.044*"trump" + 0.023*"clinton" + 0.012*"republican" + 0.009*"campaign"')
(3, u'0.008*"climate" + 0.008*"change" + 0.007*"company" + 0.007*"business"')
(4, u'0.013*"state" + 0.009*"attack" + 0.009*"military" + 0.008*"unite"')
(5, u'0.013*"obama" + 0.010*"president" + 0.008*"white" + 0.007*"people"')
(6, u'0.046*"trump" + 0.010*"president" + 0.009*"russia" + 0.009*"russian"')


In [4]:
print len(topic)

NameError: name 'topic' is not defined

In [11]:
print "hi"

hi


In [12]:
import gensim

NUM_TOPICS = 7
ldamodel = gensim.models.ldamodel.LdaModel(text_corpus, num_topics = NUM_TOPICS, id2word=text_dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

NameError: name 'text_corpus' is not defined